In [1]:
import numpy as np
import xarray as xr

import tensorflow as tf
from trainGordon_utils import subset, build_model, fullmodel, scheduler, plot_results

2023-06-20 10:02:12.166848: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-20 10:02:12.926389: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-20 10:02:17.427501: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /glade/u/apps/dav/opt/cuda/11.4.0/extras/CUPTI/lib64:/glade/u/apps/dav/opt/cuda/11.4.0/lib64:/glade/u/apps/dav/opt/openmpi/4.1.1/intel/19.1.1/lib:/glade/u/apps/dav/opt/ucx/1.11.0/lib:/glade/u/apps/opt/int

## Function Notes (trainGordon_utils.py):
<code>
# build individual models (e.g. MJO and ENSO models)
build_model(seed, # seed to initialize weights
            dropout_rate, # dropout rate (applied between input & first dense)
            activity_reg, # L2 (applied to first layer)
            hiddens, # list of nodes per each layer
            input_shape, # input shape 
            name, # input layer name
            biasbool=True # True = include bias term, False = no bias 
            )
</code>
<code>
# linearly combine MJO and ENSO models for final prediction
fullmodel(model1, # model 1
          model2, # model 2
          input1, # input into model 1 (model1)
          input2, # input into model 2 (model2)
          seed, # seed to initialize weights
          )
</code>

In [ ]:
# any additional functions: ?

## Load & Prep Data

In [2]:
X1_DIR = '' #input
X2_DIR = '' #input
Y_DIR  = '' #output

X1TRAIN_FINAME = ''
X2TRAIN_FINAME = ''
YTRAIN_FINAME  = ''

In [ ]:
# predictors [time]
X1train = xr.open_dataset(X1_DIR+X1TRAIN_FINAME)['VARNAME_HERE']

X2trainRMM1 = xr.open_dataset(X2_DIR+X2TRAIN_FINAME)['VARNAME_HERE']
X2trainRMM2 = xr.open_dataset(X2_DIR+X2TRAIN_FINAME)['VARNAME_HERE']
X2train = xr.concat([X2trainRMM1,X2trainRMM2], dim = 'new_dim') # 2xtime

# predictand [time]
Ytrain = xr.open_dataset(Y_DIR+YTRAIN_FINAME)['VARNAME_HERE']

# repeat for validation data
# ----- code here -----

In [ ]:
# lead
LEAD = 7 #days

# end X early, so we don't run out of Y data
X1train = X1train[:-1*LEAD] 
X2train = X2train[:,:-1*LEAD] 

# shift Y to account for lead
Ytrain = Ytrain[LEAD:]


# repeat for validation data
# ----- code here -----

In [ ]:
# check that Ytrain and X1/2train are the same size
# ----- code here -----

In [ ]:
# NOTE: we standardize training, validation and testing using the TRAINING mean & std (or median)

# standardize Xs using Xtrain
# ----- code here -----

# preprocess Ys by subtracting Ytrain median
# ----- code here -----

# turn Ys into 0s and 1s
# ----- code here -----

In [ ]:
# subset predictand (and predictors) to same number of 0s and 1s
    # if you subtract Ytrain median from Ytrain, you will have balanced classes, by definition
    # therefore, you only need to subset validation (& eventually testing) data to have balanced classes

n_valzero = np.shape(np.where(Yval==0)[0])[0]
n_valone  = np.shape(np.where(Yval==1)[0])[0]
i_valzero = np.where(Yval==0)[0]
i_valone  = np.where(Yval==1)[0]

# look into the "subset" function and learn how it works
X1val, Yval, i_valnew = subset(X1val, Yval, n_valzero, n_valone, i_valzero, i_valone)
X2val = X2.isel(time = i_valnew,drop=True)

In [ ]:
# convert data from xarray to numpy (xarray takes ALOT longer to train with than numpy)
# ----- code here -----
# Example: X1_val = X1val.values

## Train Model

In [1]:
# variables:
# As a good starting point, look at Gordon et al. (in prep) to identify what values she uses for these parameters 
# NOTE - we are inputing a value, not a map, so we likely need a much smaller network
SEED = 1
DROPOUT_RATE = .5

MODELNAME1 = 'ENSO'
RIDGE1 = 1
HIDDENS1 = [100,10]
INPUT_SHAPE1 = 1

MODELNAME2 = 'MJO'
RIDGE2 = .5
HIDDENS2 = [50,10]
INPUT_SHAPE2 = 1

BATCH_SIZE = 128
N_EPOCHS = 10000
PATIENCE = 20 # number of epochs of no "improvement" before training is stopped
LR = 0.001 # learning rate

In [ ]:
# ENSO MODEL
model1, input1 = build_model(SEED,
                             DROPOUT_RATE,
                             RIDGE1,
                             HIDDENS1,
                             INPUT_SHAPE1,
                             MODELNAME1)


# MJO MODEL
model2, input2 = build_model(SEED,
                             DROPOUT_RATE,
                             RIDGE2,
                             HIDDENS2,
                             INPUT_SHAPE2,
                             MODELNAME2)   

# COMBINE ENSO & MJO MODEL
model = fullmodel(model1, model2,
                  input1, input2,
                  seed)

In [ ]:
# ------ Training Hyperparameters ------
optimizer = tf.optimizers.Adam(learning_rate = LR,)
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name="sparse_categorical_accuracy", dtype=None)]                             

# ------ Compile Model -----
model.compile(optimizer = optimizer,
              loss = loss_func,
              metrics = metrics)

# ----- Callbacks -----
ES = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', mode = 'auto',
                                      patience = PATIENCE, verbose = 0, restore_best_weights = True)
LR = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose=0)
    

history = model.fit({model1name:X1_train,
                     model2name:X2_train}, 
                    Y_train, 
                    batch_size = BATCH_SIZE, 
                    epochs = N_EPOCHS, 
                    validation_data = ({model1name:X1_val,
                                        model2name:X2_val},
                                       Yval),  
                    verbose = 0,
                    callbacks=[ES,LR],
                    )

In [ ]:
#----- PLOT THE RESULTS -----
plot_results(
    history,
    exp_info=(N_EPOCHS[:100], PATIENCE),
    showplot=True
) 